<a href="https://colab.research.google.com/github/zoro1324/Air-Quality-Checker-/blob/main/Data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openaq
!pip install s3fs

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached s3fs-2025.9.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached aiobotocore-2.24.2-py3-none-any.whl.metadata (25 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-win_amd64.whl.metadata (7.9 kB)
  Using cached aioitertools-0.12.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached botocore-1.40.18-py3-none-any.whl.metadata (5.7 kB)
  Using cached multidict-6.6.4-cp313-cp313-win_amd64.whl.metadata (5.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached propcache-0.3.2-cp313-cp313-win_amd64.whl.metadata (12 kB)
  Using ca

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.40.39 requires botocore<1.41.0,>=1.40.39, but you have botocore 1.40.18 which is incompatible.


In [2]:
from openaq import OpenAQ
import pandas as pd
import s3fs

In [3]:
client = OpenAQ(api_key='1e8d7cdfee4a3c725868b9148839b5323da06cb794cac768098de81167289e3d')
client

In [4]:
sa_bbox_search_locations = client.locations.list(
bbox=(76.1667,7.9119,80.8167,13.6453),
limit=1000
)

In [5]:
sa_bbox_search_locations.results[0].__dict__

{'id': 378,
 'name': 'Alandur Bus Depot',
 'locality': None,
 'timezone': 'Asia/Kolkata',
 'country': CountryBase(id=9, code='IN', name='India'),
 'owner': OwnerBase(id=4, name='Unknown Governmental Organization'),
 'provider': ProviderBase(id=168, name='CPCB'),
 'is_mobile': False,
 'is_monitor': True,
 'instruments': [InstrumentBase(id=2, name='Government Monitor')],
 'sensors': [SensorBase(id=661, name='co µg/m³', parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name='CO mass')),
  SensorBase(id=5374, name='no2 µg/m³', parameter=ParameterBase(id=5, name='no2', units='µg/m³', display_name='NO₂ mass')),
  SensorBase(id=5352, name='o3 µg/m³', parameter=ParameterBase(id=3, name='o3', units='µg/m³', display_name='O₃ mass')),
  SensorBase(id=14304, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5')),
  SensorBase(id=14305, name='so2 µg/m³', parameter=ParameterBase(id=6, name='so2', units='µg/m³', display_name='SO₂ mass'))],
 '

In [ ]:
locations_id = {}

for location in sa_bbox_search_locations.results:
    locations_id[location.id] = location.name

len(locations_id)

84

In [ ]:
locations_id

{378: 'Alandur Bus Depot',
 412: 'Peenya, Bengaluru - KSPCB',
 594: 'BTM Layout, Bengaluru - KSPCB',
 797: 'City Railway Station - KSPCB',
 2461: 'US Diplomatic Post: Chennai',
 2549: 'IIT',
 2586: 'Manali, Chennai - CPCB',
 2589: 'SaneguravaHalli - KSPCB',
 2592: 'BWSSB Kadabesanahalli, Bengaluru - KSPCB',
 5547: 'BWSSB Kadabesanahalli, Bengaluru - CPCB',
 5548: 'BTM Layout, Bengaluru - CPCB',
 5558: 'Alandur Bus Depot, Chennai - CPCB',
 5574: 'City Railway Station, Bengaluru - KSPCB',
 5607: 'Peenya, Bengaluru - CPCB',
 5644: 'Sanegurava Halli, Bengaluru - KSPCB',
 5646: 'Plammoodu, Thiruvananthapuram - Kerala PCB',
 5655: 'Velachery Res. Area, Chennai - CPCB',
 6920: 'Manali Village, Chennai - TNPCB',
 6954: 'Chikkaballapur Rural, Chikkaballapur - KSPCB',
 6966: 'Udyogamandal, Eloor - Kerala PCB',
 6973: 'Jayanagar 5th Block, Bengaluru - KSPCB',
 6974: 'Bapuji Nagar, Bengaluru - KSPCB',
 6975: 'Silk Board, Bengaluru - KSPCB',
 6983: 'Hombegowda Nagar, Bengaluru - KSPCB',
 6984: 'Heb

In [ ]:
all_measurements_df = pd.DataFrame()

In [ ]:
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
# Create an empty DataFrame to store all measurements
all_measurements_df = pd.DataFrame()

# Define the year and the range of months
year = 2025
months = range(1, 10)  # Months from 1 (January) to 9 (September)
count = 0
for location_id in locations_id.keys():
    count+=1
    print(f"{count} . {locations_id[location_id]}: {location_id}")

    location_data_frames = [] # To store dataframes for each month for the current location

    for month in months:
        # Format the month as a two-digit string
        month_str = f"{month:02d}"

        # Construct the full S3 path for the current year and month
        full_path = f"openaq-data-archive/records/csv.gz/locationid={location_id}/year={year}/month={month_str}/*.csv.gz"

        try:
            # Use fs.glob() to find all .csv.gz files matching the constructed path
            files = fs.glob(f"s3://{full_path}")

            if files:
                # print(f"Found {len(files)} files for location ID {location_id} in {year}-{month_str}")
                # Read all files and concatenate for the current month
                month_df = pd.concat([pd.read_csv(f"s3://{file}", storage_options={"anon": True}) for file in files])
                location_data_frames.append(month_df)
            # else:
            #     print(f"No files found for location ID {location_id} for {year}-{month_str}")

        except Exception as e:
            print(f"Error accessing path for location ID {location_id} for {year}-{month_str}: {e}")


    # Concatenate all monthly dataframes for the current location
    if location_data_frames:
        location_df = pd.concat(location_data_frames, ignore_index=True)
        # Append the data for the current location to the main DataFrame
        all_measurements_df = pd.concat([all_measurements_df, location_df], ignore_index=True)
    else:
        print(f"No data found for location ID {location_id} across specified months.")

1 . Alandur Bus Depot: 378
No data found for location ID 378 across specified months.
2 . Peenya, Bengaluru - KSPCB: 412
No data found for location ID 412 across specified months.
3 . BTM Layout, Bengaluru - KSPCB: 594
No data found for location ID 594 across specified months.
4 . City Railway Station - KSPCB: 797
No data found for location ID 797 across specified months.
5 . US Diplomatic Post: Chennai: 2461
No data found for location ID 2461 across specified months.
6 . IIT: 2549
No data found for location ID 2549 across specified months.
7 . Manali, Chennai - CPCB: 2586
8 . SaneguravaHalli - KSPCB: 2589
No data found for location ID 2589 across specified months.
9 . BWSSB Kadabesanahalli, Bengaluru - KSPCB: 2592
No data found for location ID 2592 across specified months.
10 . BWSSB Kadabesanahalli, Bengaluru - CPCB: 5547
No data found for location ID 5547 across specified months.
11 . BTM Layout, Bengaluru - CPCB: 5548
12 . Alandur Bus Depot, Chennai - CPCB: 5558
No data found for l

In [ ]:
all_measurements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7177713 entries, 0 to 7177712
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   location_id  int64  
 1   sensors_id   int64  
 2   location     object 
 3   datetime     object 
 4   lat          float64
 5   lon          float64
 6   parameter    object 
 7   units        object 
 8   value        float64
dtypes: float64(3), int64(2), object(4)
memory usage: 492.9+ MB


In [ ]:
unique_locations = all_measurements_df['location'].unique()
print("Unique locations in the DataFrame:")
for loc in unique_locations:
    print(loc)

Unique locations in the DataFrame:
Manali, Chennai - CPCB-3379580
BTM Layout, Bengaluru - CPCB-3379546
City Railway Station, Bengaluru - KSPCB-3379548
Peenya, Bengaluru - CPCB-3379547
Sanegurava Halli, Bengaluru - KSPCB-3379549
Plammoodu, Thiruvananthapuram - Kerala PCB-3379553
Velachery Res. Area, Chennai - CPCB-3379566
Chikkaballapur Rural, Chikkaballapur - KSPCB-3379536
Udyogamandal, Eloor - Kerala PCB-3379604
Jayanagar 5th Block, Bengaluru - KSPCB-3379535
Bapuji Nagar, Bengaluru - KSPCB-3379532
Silk Board, Bengaluru - KSPCB-3379537
Hombegowda Nagar, Bengaluru - KSPCB-3379534
Hebbal, Bengaluru - KSPCB-3379533
Hebbal 1st Stage, Mysuru - KSPCB-3382758
Corporation Ground, Thrissur- Kerala PCB-3379663
Urban, Chamarajanagar - KSPCB-3379618
Kariavattom, Thiruvananthapuram - Kerala PCB-3379643
Chennai-931708
US Diplomatic Post: Chennai-931708
SIDCO Kurichi, Coimbatore - TNPCB-3379599
Manali Village, Chennai - TNPCB-3379598
Vijay Nagar, Ramanagara - KSPCB-3379630
Polayathode, Kollam - Keral

In [ ]:
# Filter the DataFrame to keep only locations containing "TNPCB", "chennai", or "PPCC"
filtered_df = all_measurements_df[
    all_measurements_df['location'].str.contains('TNPCB', case=False, na=False) |
    all_measurements_df['location'].str.contains('chennai', case=False, na=False) |
    all_measurements_df['location'].str.contains('PPCC', case=False, na=False)
]

# Display the filtered DataFrame information
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4186568 entries, 0 to 7177712
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   location_id  int64  
 1   sensors_id   int64  
 2   location     object 
 3   datetime     object 
 4   lat          float64
 5   lon          float64
 6   parameter    object 
 7   units        object 
 8   value        float64
dtypes: float64(3), int64(2), object(4)
memory usage: 319.4+ MB


In [ ]:
unique_locations = filtered_df['location'].unique()
print("Unique locations in the DataFrame:")
index = 0
for loc in unique_locations:
    index+=1
    print(f"{index}-",loc)

Unique locations in the DataFrame:
1- Manali, Chennai - CPCB-3379580
2- Velachery Res. Area, Chennai - CPCB-3379566
3- Chennai-931708
4- US Diplomatic Post: Chennai-931708
5- SIDCO Kurichi, Coimbatore - TNPCB-3379599
6- Manali Village, Chennai - TNPCB-3379598
7- Royapuram, Chennai - TNPCB-3379658
8- Kodungaiyur, Chennai - TNPCB-3379659
9- Meelavittan, Thoothukudi - TNPCB-3379660
10- Arumbakkam, Chennai - TNPCB-3379655
11- Perungudi, Chennai - TNPCB-3379657
12- Anthoni Pillai Nagar, Gummidipoondi - TNPCB-3379656
13- Jawahar Nagar, Puducherry - PPCC-3379683
14- Kumaran College, Tirupur - TNPCB-3379760
15- Bombay Castel, Ooty - TNPCB-3379774
16- Chalai Bazaar, Ramanathapuram - TNPCB-3379775
17- Mendonsa Colony, Dindigul - TNPCB-3379761
18- PSG College of Arts and Science, Coimbatore - TNPCB-3379773
19- Crescent University, Chengalpattu - TNPCB-3379758
20- Alandur Bus Depot, Chennai - CPCB-3379568
21- Keelapalur, Ariyalur - TNPCB-3379759
22- Municipal Corporation Office, Tirunelveli - TNPC

In [ ]:
filtered_df.to_csv('AQ_Data_TN.csv', index=False)